In [30]:
## Loading Libraries
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt 
from random import uniform
import random
from copy import deepcopy
import warnings
import pickle
import datetime
from time import gmtime, strftime
import pytz

# import power_index_calculator as px
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from generateRandSeed import generateRandSeed
from plot_simulation_output import plot_simulation_output

In [3]:
def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg

    return out

############################
def TimePolicy_generator(Graph,Maximum_Time=10000,
    method = {'name':'random','group_num':2}):
    
    # Size of Graph
    lng = Graph.number_of_nodes()
    
    Nodes_dict =  dict(Graph.nodes(data=True))
    Nodes_list = list(Nodes_dict)
    
    if method['name'] is 'random':
        # Assigning groups
        random.shuffle(Nodes_list)
        List = chunkIt(Nodes_list, method['group_num'])
        group_node = {k:v for (k,v) in zip(list(range(1,method['group_num']+1)),List)}
        
        time_node = {k:set(group_node[i%method['group_num']+1]) for (k,i) in zip(list(range(1,Maximum_Time+1)),list(range(1,Maximum_Time+1)))}
        return(time_node)
    
    elif method['name'] is 'degree':
        pass
        # # Setting Node Attributes
        # node_weight = dict(zip(Graph.nodes(),[random.randrange(10,20) for x in range(lng)]))
        # nx.set_node_attributes(Graph,node_weight,'test')
    else:
        raise ValueError('Wrong input for method["name"]!')
    

In [4]:
def Simulate_SIR_Time (Graph,Maximum_Time=100,Seasons=[50,100,150,200],P_seasons = [1,1,1,1],
                       Seeds = [0,7], n_dr=1, q_die=0.2, n_s=1, q_s=0.1, how='Transmission',
                       time_policy={'mode':'NoPlolicy','p':'','group_num':2,'method':'random'},pnt=False):
    
    """ A function to run a cascade simulation on a graph.

  The cascade is based on the SIR model in each time step.
  ....

  Parameters
  ----------
  Graph: networkx.Graph object
    The graph to run the simulation on! Each edge must have a weight
    that shows the probability of transmission.

  Maximum_Time: int
    Maximum time that the simulation is run if it does not cnoverge!

  Seasons: list of integers
    Each element shows the moment of season change.

  P_seasons: list of floats
    The Pr[tansmission] is multiplied by the p_season for each season.

  Seeds: list of integers
    Initial Seeds.
  
  n_dr: int
    Number of time steps that after infection a node dies or recovers.
  
  q_die: float
    Probability of dying at n_dr day after infection

  n_s: int
    Number of time steps that after infection a node shows symptoms.
    After showing symptoms, edge weights are multiplied by q_s.

  q_s: float
    See n_s.

  how: str
    It can be 'Transmission' or 'New_Infection'. 
    In the 'Transmission' mode, R0 is calculated as the number of virus tranmission
     (ignoring the fact that the receiver node is already infected or not).
    In the 'New_Infection' mode, R0 is calculated the number of new infected nodes,
     caused by a node in its sickness period.
  
  Attention
  ---------
  If in this simulation if we put n_dr=1, we have the simple SIR model.

  """
    # Size of Graph
    lng = Graph.number_of_nodes()

     # Checking inputs
    if n_s>n_dr:
        warnings.warn("n_s(time to show symptoms) should be less than n_dr(time to die or recovers)")

    if len(Seeds)>lng:
        raise ValueError('"numSeed" cannot be larger than Graph size')
      
    if len([x for x in Seeds if x>=lng])>lng:
        raise ValueError('Initial Seeds must be in Graph!') 
        
    if not isinstance(Graph,nx.Graph):
        raise ValueError('"Graph" must be a networkx.Graph object!')
        
    if q_die>1 or q_die<0 or q_s<0 or q_s>1 or len([x for x in P_seasons if x<0]):
        raise ValueError('Probabilities (q_s,q_die, P_seasons) must be between 0 and 1!')
        
    if how not in ['Transmission','New_Infection']:
        raise ValueError('"how" must be "Transmission" or "New_Infection"')

    # Sorting elements of Seasons
    Seasons.sort()      

    ## Creating Neighbor List
    Neighbors_List = []
    Nodes = list(Graph.nodes())
    Nodes.sort(reverse=False)
    # Finding Neighbors of each node
    for node in list(Nodes):
        Neighbors_List.append(dict(Graph.adj[node]))


    ## Initializing the Status Lists
    Normal = list(range(lng))
    Infected = list()
    Died = list()
    Recovered = list()
  
    # Adding initial seeds to infected list
    Infected = [x for x in Seeds]

    # Timer for infecteds
    Infected_timer = [0]*len(Seeds)

    # Removing infecteds from Normal list
    for node in Infected:
        Normal.remove(node)
      
    # Declaring output R0 lists
    R0_Node_list = [0]*lng
    R0_time_list = []

    # Declaring level lists
    Normal_size =[]
    Infected_size =[]
    Recovered_size =[]
    Died_size =[]
  
    # Setting first elements
    Normal_size.append(len(Normal))
    Infected_size.append(len(Seeds))
    Recovered_size.append(0)
    Died_size.append(0)
    
    
    # Activity finding
    if time_policy['mode'] is 'deterministic':
        Time_Active_dict = TimePolicy_generator(Graph,Maximum_Time=Maximum_Time,
        method = {'name':time_policy['method'],'group_num':time_policy['group_num']})
        if pnt:
            print('time_policy is {}\n\n'.format([[x,Time_Active_dict[x]] for x in range(1,time_policy['group_num']+1)]))


    # Going through time 
    for t in range(1,Maximum_Time+1):
        if pnt:
            print('************** Time {}**************'.format(t))
      
        # If there is no Infected we are on steady state. So quit!
        if len(Infected)==0:
            break
        # R_t
        R0_t = 0
    
        # Finding Season
        if  t<Seasons[0]:
            p_s = P_seasons[0]
        elif t>=Seasons[0] and t<Seasons[1]:
            p_s = P_seasons[1]
        elif t>=Seasons[1] and t<Seasons[2]:
            p_s = P_seasons[2]
        elif t>=Seasons[2] and t<Seasons[3]:
            p_s = P_seasons[3]
        else:
            p_s = 1
            

        # Finding nodes that must die or recover (based on the timer)
        DieRecover = [x for x,y in zip(Infected,Infected_timer) if y == n_dr]
        
      
        # For updating netwok, Finding nodes that their neighbor just died or . . .
        # . . . recovered so their neighbor list must be updated.                    
        nhbr_rd = set()
        # For each node in DieRecover list
        for node in DieRecover:
            # The neighbors of the dead or recoverd node need to be modified
            nhbr_rd = nhbr_rd.union(Graph.adj[node])    

            # We delete that node from infecteds and timer
            index = Infected.index(node)
            del Infected_timer[index]
            del Infected[index]

            # We remove the edges for that node (Recovereds can not get sick again)
            Graph.remove_edges_from(list(Graph.edges(node)))
          
            # We see they die or recover with respect to q_die
            if uniform(0,1) < q_die:
                # It dies
                Died.append(node)

            else:
                # It recovers
                Recovered.append(node)
      
        if pnt:
            print('DieRecover is    : ',DieRecover)
            print('Died is          : ',Died)
            print('Recovered is     : ',Recovered)
            print('Normal is        : ',Normal)
            print('Infected is      : ',Infected)
            print('Infected_timer is: ',Infected_timer)
            
        
        # Updating Neighbor List for nodes that their neighbrs are died or recoverd
        for neighbor in nhbr_rd:
            Neighbors_List[neighbor] = dict(Graph.adj[neighbor]) 
            
        
        # Determining Active nodes
        if time_policy['mode'] is 'deterministic':
            ActiveNodes = Time_Active_dict[t].intersection(set(Normal+Infected))
        elif time_policy['mode'] is 'stochastic':
            p_active = time_policy['p']
            ActiveNodes = set([x for x in Normal+Infected if uniform(0,1)<p_active])
        else:
            ActiveNodes = set(Normal+Infected)
            warnings.warn('"time_policy"["mode"] must be "deterministic" or "stochastic"!')
        if pnt:
            print('ActiveNodes is         : ',ActiveNodes)
            print('Infected*ActiveNodes is: ',set(Infected).intersection(ActiveNodes))
            
            
        # Cascade...
        # New infected nodes in each time step
        New_Infected = []
        # For each Infected node we try to find the transmissions
        for node in set(Infected).intersection(ActiveNodes):
    
            # Number of Transmission by this node in this time step
            R0 = 0
          
            # Checking if the node has symptoms
            if Infected_timer[Infected.index(node)] < n_s:
                Q = 1
            else:
                Q = q_s
                   
            ## Finding the Neighbors of the node based on the how (transmission or new infection)
            if how is 'Transmission':
                Neighbors = {k:v for k,v in Neighbors_List[node].items() if k in ActiveNodes}               

            else:
                Neighbors = {k:v for k,v in Neighbors_List[node].items() if k in Normal and k in ActiveNodes}
                
            if pnt:
                print('\n*Node {} is selected and its all neigbohrs are {} and its active neighbors are {}'.format(
                    node,list(Neighbors_List[node]),list(Neighbors)))
          
            # For each neighbor
            for advicee in Neighbors:
                # Gets informed with respect to edge probability, symptoms, season
                if  uniform(0,1)<Neighbors[advicee]['weight']*Q*p_s:
                    # Increasing the R-0 for node and time
                    R0+=1
                    R0_t+=1  
                    # If its a new node (which is not already informed) we update the list of informeds
                    if advicee in Normal:
                        Normal.remove(advicee)
                        New_Infected.append(advicee)
                        
                    if pnt:
                        print('**Node {} got informed!'.format(advicee))
                        print('**Normal is        : ',Normal)
                        print('**New_Infected is  : ',New_Infected)
                        
          

            # Updating number of new infected for this node
            R0_Node_list[node] = R0_Node_list[node]+R0
          

        ## Increasing Timer of nodes ##
        Infected_timer = [x+1 for x in Infected_timer]

        ## Updaint list of R_t
        try: 
            R0_time_list.append(R0_t/len(Infected))

        except:
            R0_time_list.append(np.nan)

        # Updating Infected list and the timer
        Infected = Infected + New_Infected
        Infected_timer = Infected_timer + len(New_Infected)*[0]
      
        # Updating level sizes in each step
        Normal_size.append(len(Normal))
        Infected_size.append(len(Infected))
        Recovered_size.append(len(Recovered))
        Died_size.append(len(Died))
        
        if t==Maximum_Time:
            raise ValueError('Increase Maximum_Time! Time ended before the end of the cascade!')
            
            
    # Keeping nodes which got infected 
    R0_Infected_Nodes = {k:R0_Node_list[k] for k in range(len(R0_Node_list)) if k not in Normal}
  
    # The level output
    level = pd.DataFrame(data={'time':list(range(len(Normal_size))),
                             'Normal':Normal_size,
                             'Infected':Infected_size,
                             'Recovered':Recovered_size,
                             'Died':Died_size})
    
    
    # Finding The Interval in Peak
    temp = list(level.Infected)
    Peak = max(temp)
    t_peak = temp.index(Peak)
    val_list = [abs(x-0.5*Peak) for x in temp]

    try:
        t1 = val_list.index(min(val_list[0:t_peak]))
        t2 = val_list[t_peak:].index(min(val_list[t_peak:]))+t_peak
        Peak_int = t2-t1
    except:
        Peak_int = np.nan
    
    
    OUT={'Level':level,'R0_Nodes':R0_Infected_Nodes,'R0':np.mean(list(R0_Infected_Nodes.values())),
         'Cascade':(len(Died)+len(Recovered)+len(Infected))/lng,'R0_time_list':[0]+R0_time_list,
         'Peak':Peak/lng,'Peak_int':Peak_int}
    return(OUT)


# **Run Your Simulation**

In the below example, we create an Erdos-Renyi graph and we run the simulation and plot the outputs.

In [58]:
%%time
# Example
n = 10
d = 3
G = nx.erdos_renyi_graph(n,d/n)
edge_weight = dict(zip(G.edges(),[uniform(0.01,0.1) for x in range(len(G.edges()))]))
nx.set_edge_attributes(G,edge_weight,'weight')

method = 'Transmission'
out = Simulate_SIR_Time (deepcopy(G),Maximum_Time=10000,Seasons=[20,40,60,80],P_seasons = [1,1,1,1],
                   Seeds = generateRandSeed(2,G), n_dr=17, q_die=0.2, n_s=5, q_s=0.1, how=method,
                        time_policy={'mode':'deterministic','p':0.5,'group_num':2,'method':'random'},pnt=True)

print('The Peak is {:0.3f}, and the Cascade is {:0.3f}, and Peak_int is {}'.format(
    out['Peak'],out['Cascade'],out['Peak_int']))
# plot_simulation_output(Graphsize=G.number_of_nodes(),data = out,module=['Infected','Died'])

time_policy is [[1, {1, 4, 5, 8, 9}], [2, {0, 2, 3, 6, 7}]]


************** Time 1**************
DieRecover is    :  []
Died is          :  []
Recovered is     :  []
Normal is        :  [0, 1, 2, 3, 5, 7, 8, 9]
Infected is      :  [4, 6]
Infected_timer is:  [0, 0]
ActiveNodes is         :  {1, 4, 5, 8, 9}
Infected*ActiveNodes is:  {4}

*Node 4 is selected and its all neigbohrs are [0, 6] and its active neighbors are []
************** Time 2**************
DieRecover is    :  []
Died is          :  []
Recovered is     :  []
Normal is        :  [0, 1, 2, 3, 5, 7, 8, 9]
Infected is      :  [4, 6]
Infected_timer is:  [1, 1]
ActiveNodes is         :  {0, 2, 3, 6, 7}
Infected*ActiveNodes is:  {6}

*Node 6 is selected and its all neigbohrs are [2, 4, 7] and its active neighbors are [2, 7]
************** Time 3**************
DieRecover is    :  []
Died is          :  []
Recovered is     :  []
Normal is        :  [0, 1, 2, 3, 5, 7, 8, 9]
Infected is      :  [4, 6]
Infected_timer is:  [2, 2]
Act

In [55]:
%%time
OUT = {'deter':{'Cascade':list(),'Peak':list(),'Peak_int':list()},
       'stoch':{'Cascade':list(),'Peak':list(),'Peak_int':list()}}
n = 2000
d = 20
for i in range(1000):
    if i%50==0:
        now = datetime.datetime.now()
        
        print ("\n{} from {}\n*Current date and time : ".format(i,1000))
        print (now.strftime("%H:%M:%S"))
        print ("\n")
    
        
        
    G = nx.erdos_renyi_graph(n,d/n)
    edge_weight = dict(zip(G.edges(),[uniform(0.01,0.1) for x in range(len(G.edges()))]))
    nx.set_edge_attributes(G,edge_weight,'weight')
    
    
    out = Simulate_SIR_Time (deepcopy(G),Maximum_Time=10000,Seasons=[200,400,600,800],P_seasons = [1,1,1,1],
                   Seeds = generateRandSeed(10,G), n_dr=17, q_die=0.2, n_s=7, q_s=0.1, how='Transmission',
                        time_policy={'mode':'deterministic','p':0.5,'group_num':2,'method':'random'},pnt=False)
    OUT['deter']['Cascade'].append(out['Cascade'])
    OUT['deter']['Peak'].append(out['Peak'])
    OUT['deter']['Peak_int'].append(out['Peak_int'])
    
    
    out = Simulate_SIR_Time (deepcopy(G),Maximum_Time=10000,Seasons=[200,400,600,800],P_seasons = [1,1,1,1],
                   Seeds = generateRandSeed(10,G), n_dr=17, q_die=0.2, n_s=7, q_s=0.1, how='Transmission',
                        time_policy={'mode':'stochastic','p':0.5,'group_num':2,'method':'random'},pnt=False)
    OUT['stoch']['Cascade'].append(out['Cascade'])
    OUT['stoch']['Peak'].append(out['Peak'])
    OUT['stoch']['Peak_int'].append(out['Peak_int'])

deter = pd.DataFrame(OUT['deter'])
stoch = pd.DataFrame(OUT['stoch'])


0 from 1000
*Current date and time : 
02:39:25



50 from 1000
*Current date and time : 
02:44:56



100 from 1000
*Current date and time : 
02:50:59



150 from 1000
*Current date and time : 
02:56:52



200 from 1000
*Current date and time : 
03:02:16



250 from 1000
*Current date and time : 
03:07:47



300 from 1000
*Current date and time : 
03:13:18



350 from 1000
*Current date and time : 
03:19:21



400 from 1000
*Current date and time : 
03:24:54



450 from 1000
*Current date and time : 
03:30:24



500 from 1000
*Current date and time : 
03:35:51



550 from 1000
*Current date and time : 
03:41:22



600 from 1000
*Current date and time : 
03:46:53



650 from 1000
*Current date and time : 
03:52:20



700 from 1000
*Current date and time : 
03:57:40



750 from 1000
*Current date and time : 
04:03:01



800 from 1000
*Current date and time : 
04:08:33



850 from 1000
*Current date and time : 
04:13:58



900 from 1000
*Current date and time : 
04:19:37



950 from 1000


In [56]:
pd.DataFrame(np.mean(deter),columns=['Mean-deter'])
pd.DataFrame(np.mean(stoch),columns=['Mean-stoch'])

deter.describe()
stoch.describe()

,Mean-deter
Cascade,0.71557
Peak,0.36555
Peak_int,33.07200


,Mean-stoch
Cascade,0.806984
Peak,0.526643
Peak_int,25.227000


,Cascade,Peak,Peak_int
count,1000.000000,1000.000000,1000.000000
mean,0.715570,0.365550,33.072000
std,0.045751,0.048884,5.638493
min,0.329000,0.160000,25.000000
25%,0.706500,0.342000,30.000000
50%,0.721000,0.372500,32.000000
75%,0.734000,0.398125,34.000000
max,0.777500,0.477500,66.000000


,Cascade,Peak,Peak_int
count,1000.000000,1000.000000,1000.000000
mean,0.806984,0.526643,25.227000
std,0.014364,0.027244,1.219409
min,0.758000,0.427500,21.000000
25%,0.797000,0.508500,24.000000
50%,0.807500,0.525500,25.000000
75%,0.817000,0.546000,26.000000
max,0.850000,0.621500,31.000000
